In [1]:
import tensorflow as tf
import sys
import tensorflow_hub as tfhb
import matplotlib.pyplot as plt

In [2]:
print(tf.__version__)
print(sys.version)
print(tf.config.list_physical_devices("GPU"))

2.10.0
3.10.0 | packaged by conda-forge | (default, Nov 10 2021, 13:20:59) [MSC v.1916 64 bit (AMD64)]
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# Helper FUNCTIONS

In [3]:
def get_model_information(model):
    print("Thickness of the model:", len(model.weights))
    print ("{:<80} {:<20} {:<10}".format('Layer','Shape','Type'))

    for i in range(len(model.weights)):
        model_weights_raw_string = str(model.weights[i])
        model_weights_wo_weights = model_weights_raw_string.split(", numpy", 1)[0]
        dtype = model_weights_wo_weights.split(" dtype=")[1]
        shape = model_weights_wo_weights.split(" shape=")[1].split(" dtype=")[0]
        print ("{:<80} {:<20} {:<10}".format(model.weights[i].name, shape, dtype))

In [24]:
import datetime

def create_tensorboard_callback(dir_name,experiment_name):
    
    log_dir = dir_name + "//" + experiment_name + "//" + datetime.datetime.now().strftime("%d_%m_%Y___%H_%M_%S")
    tensorboard_callback_obj = tf.keras.callbacks.TensorBoard(log_dir=log_dir,
                                                              histogram_freq=1,
                                                              update_freq='batch')
    print(f"Creating TensorBoard Callback in:- {log_dir}")
    return tensorboard_callback_obj    

#  Storing LINKS for Hub Models

## Feature Vector Models

In [3]:
efficient_net_b0_fv_link = "https://tfhub.dev/google/efficientnet/b0/feature-vector/1"
efficient_net_b7_fv_link = "https://tfhub.dev/google/efficientnet/b7/feature-vector/1"

## Classification Models

In [6]:
efficient_net_b0_cf_link = "https://tfhub.dev/google/efficientnet/b0/classification/1"
efficient_net_b7_cf_link = "https://tfhub.dev/google/efficientnet/b7/classification/1"

# Loading the Data Into Datasets for Models

In [7]:
train_data_dir = r"D:\Machine Learning\Sample Project1\Notes Notebooks\Working_Data\10_food_classes_10_percent\10_food_classes_10_percent\train"
test_data_dir = r"D:\Machine Learning\Sample Project1\Notes Notebooks\Working_Data\10_food_classes_10_percent\10_food_classes_10_percent\test"
class_labels = ['chicken_curry','chicken_wings','fried_rice','grilled_salmon','hamburger','ice_cream','pizza','ramen','steak','sushi']

In [8]:
train_ds_mn_224, valid_ds_mn_224 = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                                               shuffle=True,
                                                                               label_mode="categorical",
                                                                               validation_split=0.1,
                                                                               seed=42,
                                                                               subset="both",
                                                                               image_size=(224,224),
                                                                               batch_size=32)

test_ds_mn_224 = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                             shuffle=True,
                                                             label_mode="categorical",
                                                             image_size=(224,224),
                                                             batch_size=32)

Found 750 files belonging to 10 classes.
Using 675 files for training.
Using 75 files for validation.
Found 2500 files belonging to 10 classes.


In [7]:
train_ds_mn_600, valid_ds_mn_600 = tf.keras.utils.image_dataset_from_directory(train_data_dir,
                                                                               label_mode="categorical",
                                                                               shuffle=True,
                                                                               validation_split=0.1,
                                                                               seed=42,
                                                                               subset="both",
                                                                               image_size=(600,600),
                                                                               batch_size=32)

test_ds_mn_600 = tf.keras.utils.image_dataset_from_directory(test_data_dir,
                                                             label_mode="categorical",
                                                             shuffle=True,
                                                             image_size=(600,600),
                                                             batch_size=32)

Found 750 files belonging to 10 classes.
Using 675 files for training.
Using 75 files for validation.
Found 2500 files belonging to 10 classes.


# Training and Fitting On Efficient Net B0 Model

## Feature Vector Model

In [25]:
efficient_net_b0_fv_base_model = tfhb.KerasLayer(efficient_net_b0_fv_link,
                                                 trainable=False)

In [26]:
get_model_information(efficient_net_b0_fv_base_model)

Thickness of the model: 309
Layer                                                                            Shape                Type      
efficientnet-b0/stem/conv2d/kernel:0                                             (3, 3, 3, 32)        float32   
efficientnet-b0/stem/tpu_batch_normalization/gamma:0                             (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/beta:0                              (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/moving_mean:0                       (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/moving_variance:0                   (32,)                float32   
efficientnet-b0/blocks_0/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 32, 1)        float32   
efficientnet-b0/blocks_0/tpu_batch_normalization/gamma:0                         (32,)                float32   
efficientnet-b0/blocks_0/tpu_batch_normalization/beta:0             

efficientnet-b0/blocks_6/se/conv2d/kernel:0                                      (1, 1, 480, 20)      float32   
efficientnet-b0/blocks_6/se/conv2d/bias:0                                        (20,)                float32   
efficientnet-b0/blocks_6/se/conv2d_1/kernel:0                                    (1, 1, 20, 480)      float32   
efficientnet-b0/blocks_6/se/conv2d_1/bias:0                                      (480,)               float32   
efficientnet-b0/blocks_6/conv2d_1/kernel:0                                       (1, 1, 480, 80)      float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/gamma:0                       (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/beta:0                        (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/moving_mean:0                 (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/moving_variance:0             (80,)          

efficientnet-b0/blocks_11/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 672, 1)       float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/gamma:0                      (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/beta:0                       (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/moving_mean:0                (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/moving_variance:0            (672,)               float32   
efficientnet-b0/blocks_11/se/conv2d/kernel:0                                     (1, 1, 672, 28)      float32   
efficientnet-b0/blocks_11/se/conv2d/bias:0                                       (28,)                float32   
efficientnet-b0/blocks_11/se/conv2d_1/kernel:0                                   (1, 1, 28, 672)      float32   
efficientnet-b0/blocks_11/se/conv2d_1/bias:0                                     (672,)         

In [27]:
efficient_net_b0_fv_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b0_fv_base_model,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b0_fv_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [28]:
efficient_net_b0_fv_model.fit(train_ds_mn_224,
                              epochs=10,
                              steps_per_epoch=len(train_ds_mn_224),
                              validation_data=valid_ds_mn_224,
                              validation_steps=len(valid_ds_mn_224),
                              callbacks=[create_tensorboard_callback(dir_name="TFModels",
                                                                     experiment_name="Efficient_Net_fv_b0")])

Creating TensorBoard Callback in:- TFModels//Efficient_Net_fv_b0//18_07_2023___19_37_00
Epoch 1/10
22/22 [==============================] - 8s 247ms/step - loss: 1.8891 - accuracy: 0.3837 - val_loss: 1.5003 - val_accuracy: 0.6133
Epoch 2/10
22/22 [==============================] - 4s 191ms/step - loss: 1.0978 - accuracy: 0.7704 - val_loss: 1.1092 - val_accuracy: 0.7067
Epoch 3/10
22/22 [==============================] - 4s 193ms/step - loss: 0.7782 - accuracy: 0.8326 - val_loss: 0.9503 - val_accuracy: 0.7467
Epoch 4/10
22/22 [==============================] - 4s 191ms/step - loss: 0.6251 - accuracy: 0.8711 - val_loss: 0.8438 - val_accuracy: 0.7733
Epoch 5/10
22/22 [==============================] - 4s 195ms/step - loss: 0.5290 - accuracy: 0.8933 - val_loss: 0.7960 - val_accuracy: 0.7867
Epoch 6/10
22/22 [==============================] - 4s 178ms/step - loss: 0.4611 - accuracy: 0.9111 - val_loss: 0.7599 - val_accuracy: 0.8000
Epoch 7/10
22/22 [==============================] - 4s 180ms

# Classification Model

In [20]:
efficient_net_b0_cf_base_model = tfhb.KerasLayer(efficient_net_b0_cf_link,
                                                 trainable=False)

In [21]:
get_model_information(efficient_net_b0_cf_base_model)

Thickness of the model: 311
Layer                                                                            Shape                Type      
efficientnet-b0/stem/conv2d/kernel:0                                             (3, 3, 3, 32)        float32   
efficientnet-b0/stem/tpu_batch_normalization/gamma:0                             (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/beta:0                              (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/moving_mean:0                       (32,)                float32   
efficientnet-b0/stem/tpu_batch_normalization/moving_variance:0                   (32,)                float32   
efficientnet-b0/blocks_0/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 32, 1)        float32   
efficientnet-b0/blocks_0/tpu_batch_normalization/gamma:0                         (32,)                float32   
efficientnet-b0/blocks_0/tpu_batch_normalization/beta:0             

efficientnet-b0/blocks_6/se/conv2d_1/bias:0                                      (480,)               float32   
efficientnet-b0/blocks_6/conv2d_1/kernel:0                                       (1, 1, 480, 80)      float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/gamma:0                       (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/beta:0                        (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/moving_mean:0                 (80,)                float32   
efficientnet-b0/blocks_6/tpu_batch_normalization_2/moving_variance:0             (80,)                float32   
efficientnet-b0/blocks_7/conv2d/kernel:0                                         (1, 1, 80, 480)      float32   
efficientnet-b0/blocks_7/tpu_batch_normalization/gamma:0                         (480,)               float32   
efficientnet-b0/blocks_7/tpu_batch_normalization/beta:0                          (480,)         

efficientnet-b0/blocks_11/tpu_batch_normalization_1/gamma:0                      (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/beta:0                       (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/moving_mean:0                (672,)               float32   
efficientnet-b0/blocks_11/tpu_batch_normalization_1/moving_variance:0            (672,)               float32   
efficientnet-b0/blocks_11/se/conv2d/kernel:0                                     (1, 1, 672, 28)      float32   
efficientnet-b0/blocks_11/se/conv2d/bias:0                                       (28,)                float32   
efficientnet-b0/blocks_11/se/conv2d_1/kernel:0                                   (1, 1, 28, 672)      float32   
efficientnet-b0/blocks_11/se/conv2d_1/bias:0                                     (672,)               float32   
efficientnet-b0/blocks_11/conv2d_1/kernel:0                                      (1, 1, 672, 192

In [22]:
efficient_net_b0_cf_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                       efficient_net_b0_cf_base_model,
                                                       tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b0_cf_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [23]:
efficient_net_b0_cf_model.fit(train_ds_mn_224,
                              epochs=10,
                              steps_per_epoch=len(train_ds_mn_224),
                              validation_data=valid_ds_mn_224,
                              validation_steps=len(valid_ds_mn_224),
                              callbacks=[create_tensorboard_callback(dir_name="TFModels",
                                                                     experiment_name="efficient_net_b0_cf")])

Creating TensorBoard Callback in:- TFModels//efficient_net_b0_cf//18_07_2023___19_34_28
Epoch 1/10
22/22 [==============================] - 6s 104ms/step - loss: 2.1146 - accuracy: 0.3556 - val_loss: 1.3031 - val_accuracy: 0.6533
Epoch 2/10
22/22 [==============================] - 2s 65ms/step - loss: 0.8019 - accuracy: 0.7644 - val_loss: 0.9733 - val_accuracy: 0.7200
Epoch 3/10
22/22 [==============================] - 2s 66ms/step - loss: 0.5240 - accuracy: 0.8385 - val_loss: 0.9243 - val_accuracy: 0.7467
Epoch 4/10
22/22 [==============================] - 2s 110ms/step - loss: 0.4075 - accuracy: 0.9022 - val_loss: 0.8856 - val_accuracy: 0.7467
Epoch 5/10
22/22 [==============================] - 2s 69ms/step - loss: 0.3295 - accuracy: 0.9289 - val_loss: 0.8728 - val_accuracy: 0.7600
Epoch 6/10
22/22 [==============================] - 2s 68ms/step - loss: 0.2806 - accuracy: 0.9467 - val_loss: 0.8359 - val_accuracy: 0.7733
Epoch 7/10
22/22 [==============================] - 2s 68ms/step

# Training and Fitting Efficient Net B7 Model

## Feature Vector Model ON 600 * 600 images

In [4]:
efficient_net_b7_fv_base_model = tfhb.KerasLayer(efficient_net_b7_fv_link,
                                                 trainable=False)

ValueError: Trying to load a model of incompatible/unknown type. 'C:\Users\pc\AppData\Local\Temp\tfhub_modules\12ac9e16804c41d517c7da18f23e17fd921d9d48' contains neither 'saved_model.pb' nor 'saved_model.pbtxt'.

In [50]:
get_model_information(efficient_net_b7_fv_base_model)

Thickness of the model: 1035
Layer                                                                            Shape                Type      
efficientnet-b7/stem/conv2d/kernel:0                                             (3, 3, 3, 64)        float32   
efficientnet-b7/stem/tpu_batch_normalization/gamma:0                             (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/beta:0                              (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/moving_mean:0                       (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/moving_variance:0                   (64,)                float32   
efficientnet-b7/blocks_0/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 64, 1)        float32   
efficientnet-b7/blocks_0/tpu_batch_normalization/gamma:0                         (64,)                float32   
efficientnet-b7/blocks_0/tpu_batch_normalization/beta:0            

efficientnet-b7/blocks_7/tpu_batch_normalization/beta:0                          (288,)               float32   
efficientnet-b7/blocks_7/tpu_batch_normalization/moving_mean:0                   (288,)               float32   
efficientnet-b7/blocks_7/tpu_batch_normalization/moving_variance:0               (288,)               float32   
efficientnet-b7/blocks_7/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 288, 1)       float32   
efficientnet-b7/blocks_7/tpu_batch_normalization_1/gamma:0                       (288,)               float32   
efficientnet-b7/blocks_7/tpu_batch_normalization_1/beta:0                        (288,)               float32   
efficientnet-b7/blocks_7/tpu_batch_normalization_1/moving_mean:0                 (288,)               float32   
efficientnet-b7/blocks_7/tpu_batch_normalization_1/moving_variance:0             (288,)               float32   
efficientnet-b7/blocks_7/se/conv2d/kernel:0                                      (1, 1, 288, 12)

efficientnet-b7/blocks_12/tpu_batch_normalization_1/moving_mean:0                (480,)               float32   
efficientnet-b7/blocks_12/tpu_batch_normalization_1/moving_variance:0            (480,)               float32   
efficientnet-b7/blocks_12/se/conv2d/kernel:0                                     (1, 1, 480, 20)      float32   
efficientnet-b7/blocks_12/se/conv2d/bias:0                                       (20,)                float32   
efficientnet-b7/blocks_12/se/conv2d_1/kernel:0                                   (1, 1, 20, 480)      float32   
efficientnet-b7/blocks_12/se/conv2d_1/bias:0                                     (480,)               float32   
efficientnet-b7/blocks_12/conv2d_1/kernel:0                                      (1, 1, 480, 80)      float32   
efficientnet-b7/blocks_12/tpu_batch_normalization_2/gamma:0                      (80,)                float32   
efficientnet-b7/blocks_12/tpu_batch_normalization_2/beta:0                       (80,)          

efficientnet-b7/blocks_17/tpu_batch_normalization/moving_variance:0              (480,)               float32   
efficientnet-b7/blocks_17/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 480, 1)       float32   
efficientnet-b7/blocks_17/tpu_batch_normalization_1/gamma:0                      (480,)               float32   
efficientnet-b7/blocks_17/tpu_batch_normalization_1/beta:0                       (480,)               float32   
efficientnet-b7/blocks_17/tpu_batch_normalization_1/moving_mean:0                (480,)               float32   
efficientnet-b7/blocks_17/tpu_batch_normalization_1/moving_variance:0            (480,)               float32   
efficientnet-b7/blocks_17/se/conv2d/kernel:0                                     (1, 1, 480, 20)      float32   
efficientnet-b7/blocks_17/se/conv2d/bias:0                                       (20,)                float32   
efficientnet-b7/blocks_17/se/conv2d_1/kernel:0                                   (1, 1, 20, 480)

efficientnet-b7/blocks_21/tpu_batch_normalization/beta:0                         (960,)               float32   
efficientnet-b7/blocks_21/tpu_batch_normalization/moving_mean:0                  (960,)               float32   
efficientnet-b7/blocks_21/tpu_batch_normalization/moving_variance:0              (960,)               float32   
efficientnet-b7/blocks_21/depthwise_conv2d/depthwise_kernel:0                    (3, 3, 960, 1)       float32   
efficientnet-b7/blocks_21/tpu_batch_normalization_1/gamma:0                      (960,)               float32   
efficientnet-b7/blocks_21/tpu_batch_normalization_1/beta:0                       (960,)               float32   
efficientnet-b7/blocks_21/tpu_batch_normalization_1/moving_mean:0                (960,)               float32   
efficientnet-b7/blocks_21/tpu_batch_normalization_1/moving_variance:0            (960,)               float32   
efficientnet-b7/blocks_21/se/conv2d/kernel:0                                     (1, 1, 960, 40)

...

efficientnet-b7/blocks_32/tpu_batch_normalization/moving_variance:0              (1344,)              float32   
efficientnet-b7/blocks_32/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 1344, 1)      float32   
efficientnet-b7/blocks_32/tpu_batch_normalization_1/gamma:0                      (1344,)              float32   
efficientnet-b7/blocks_32/tpu_batch_normalization_1/beta:0                       (1344,)              float32   
efficientnet-b7/blocks_32/tpu_batch_normalization_1/moving_mean:0                (1344,)              float32   
efficientnet-b7/blocks_32/tpu_batch_normalization_1/moving_variance:0            (1344,)              float32   
efficientnet-b7/blocks_32/se/conv2d/kernel:0                                     (1, 1, 1344, 56)     float32   
efficientnet-b7/blocks_32/se/conv2d/bias:0                                       (56,)                float32   
efficientnet-b7/blocks_32/se/conv2d_1/kernel:0                                   (1, 1, 56, 1344

efficientnet-b7/blocks_38/tpu_batch_normalization_2/beta:0                       (384,)               float32   
efficientnet-b7/blocks_38/tpu_batch_normalization_2/moving_mean:0                (384,)               float32   
efficientnet-b7/blocks_38/tpu_batch_normalization_2/moving_variance:0            (384,)               float32   
efficientnet-b7/blocks_39/conv2d/kernel:0                                        (1, 1, 384, 2304)    float32   
efficientnet-b7/blocks_39/tpu_batch_normalization/gamma:0                        (2304,)              float32   
efficientnet-b7/blocks_39/tpu_batch_normalization/beta:0                         (2304,)              float32   
efficientnet-b7/blocks_39/tpu_batch_normalization/moving_mean:0                  (2304,)              float32   
efficientnet-b7/blocks_39/tpu_batch_normalization/moving_variance:0              (2304,)              float32   
efficientnet-b7/blocks_39/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 2304, 1)

efficientnet-b7/blocks_44/tpu_batch_normalization_1/beta:0                       (2304,)              float32   
efficientnet-b7/blocks_44/tpu_batch_normalization_1/moving_mean:0                (2304,)              float32   
efficientnet-b7/blocks_44/tpu_batch_normalization_1/moving_variance:0            (2304,)              float32   
efficientnet-b7/blocks_44/se/conv2d/kernel:0                                     (1, 1, 2304, 96)     float32   
efficientnet-b7/blocks_44/se/conv2d/bias:0                                       (96,)                float32   
efficientnet-b7/blocks_44/se/conv2d_1/kernel:0                                   (1, 1, 96, 2304)     float32   
efficientnet-b7/blocks_44/se/conv2d_1/bias:0                                     (2304,)              float32   
efficientnet-b7/blocks_44/conv2d_1/kernel:0                                      (1, 1, 2304, 384)    float32   
efficientnet-b7/blocks_44/tpu_batch_normalization_2/gamma:0                      (384,)         

efficientnet-b7/blocks_50/conv2d/kernel:0                                        (1, 1, 384, 2304)    float32   
efficientnet-b7/blocks_50/tpu_batch_normalization/gamma:0                        (2304,)              float32   
efficientnet-b7/blocks_50/tpu_batch_normalization/beta:0                         (2304,)              float32   
efficientnet-b7/blocks_50/tpu_batch_normalization/moving_mean:0                  (2304,)              float32   
efficientnet-b7/blocks_50/tpu_batch_normalization/moving_variance:0              (2304,)              float32   
efficientnet-b7/blocks_50/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 2304, 1)      float32   
efficientnet-b7/blocks_50/tpu_batch_normalization_1/gamma:0                      (2304,)              float32   
efficientnet-b7/blocks_50/tpu_batch_normalization_1/beta:0                       (2304,)              float32   
efficientnet-b7/blocks_50/tpu_batch_normalization_1/moving_mean:0                (2304,)        

efficientnet-b7/blocks_54/tpu_batch_normalization_2/moving_mean:0                (640,)               float32   
efficientnet-b7/blocks_54/tpu_batch_normalization_2/moving_variance:0            (640,)               float32   
efficientnet-b7/head/conv2d/kernel:0                                             (1, 1, 640, 2560)    float32   
efficientnet-b7/head/tpu_batch_normalization/gamma:0                             (2560,)              float32   
efficientnet-b7/head/tpu_batch_normalization/beta:0                              (2560,)              float32   
efficientnet-b7/head/tpu_batch_normalization/moving_mean:0                       (2560,)              float32   
efficientnet-b7/head/tpu_batch_normalization/moving_variance:0                   (2560,)              float32   


In [53]:
efficient_net_b7_fv_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b7_fv_base_model,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b7_fv_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [54]:
efficient_net_b7_fv_model.fit(train_ds_mn_600,
                              epochs=10,
                              steps_per_epoch=len(train_ds_mn_600),
                              validation_data=valid_ds_mn_600,
                              validation_steps=len(valid_ds_mn_600))

Epoch 1/10
22/22 [==============================] - 50s 2s/step - loss: 1.5994 - accuracy: 0.6163 - val_loss: 1.0190 - val_accuracy: 0.7733
Epoch 2/10
22/22 [==============================] - 38s 2s/step - loss: 0.7631 - accuracy: 0.8444 - val_loss: 0.7026 - val_accuracy: 0.8267
Epoch 3/10
22/22 [==============================] - 39s 2s/step - loss: 0.5355 - accuracy: 0.8904 - val_loss: 0.5842 - val_accuracy: 0.8400
Epoch 4/10
22/22 [==============================] - 40s 2s/step - loss: 0.4284 - accuracy: 0.9052 - val_loss: 0.5243 - val_accuracy: 0.8667
Epoch 5/10
22/22 [==============================] - 40s 2s/step - loss: 0.3686 - accuracy: 0.9230 - val_loss: 0.4856 - val_accuracy: 0.8667
Epoch 6/10
22/22 [==============================] - 41s 2s/step - loss: 0.3218 - accuracy: 0.9319 - val_loss: 0.4508 - val_accuracy: 0.8667
Epoch 7/10
22/22 [==============================] - 41s 2s/step - loss: 0.2845 - accuracy: 0.9393 - val_loss: 0.4314 - val_accuracy: 0.8667
Epoch 8/10
22/22 [==

## Feature Vector Model on 224 * 224 images

In [61]:
efficient_net_b7_fv_base_model_fg = tfhb.KerasLayer(efficient_net_b7_fv_link,
                                                 trainable=False)

In [62]:
efficient_net_b7_fv_model_fg = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b7_fv_base_model_fg,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b7_fv_model_fg.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [63]:
efficient_net_b7_fv_model_fg.fit(train_ds_mn_224,
                                 epochs=10,
                                 steps_per_epoch=len(train_ds_mn_224),
                                 validation_data=valid_ds_mn_224,
                                 validation_steps=len(valid_ds_mn_224))

Epoch 1/10
22/22 [==============================] - 19s 417ms/step - loss: 1.6399 - accuracy: 0.5467 - val_loss: 1.1537 - val_accuracy: 0.6933
Epoch 2/10
22/22 [==============================] - 6s 274ms/step - loss: 0.8101 - accuracy: 0.8356 - val_loss: 0.9130 - val_accuracy: 0.7200
Epoch 3/10
22/22 [==============================] - 6s 276ms/step - loss: 0.5930 - accuracy: 0.8681 - val_loss: 0.8332 - val_accuracy: 0.7333
Epoch 4/10
22/22 [==============================] - 6s 280ms/step - loss: 0.4850 - accuracy: 0.8948 - val_loss: 0.8079 - val_accuracy: 0.7600
Epoch 5/10
22/22 [==============================] - 6s 277ms/step - loss: 0.4198 - accuracy: 0.9096 - val_loss: 0.7743 - val_accuracy: 0.8000
Epoch 6/10
22/22 [==============================] - 6s 282ms/step - loss: 0.3657 - accuracy: 0.9244 - val_loss: 0.7573 - val_accuracy: 0.7867
Epoch 7/10
22/22 [==============================] - 6s 276ms/step - loss: 0.3224 - accuracy: 0.9319 - val_loss: 0.7545 - val_accuracy: 0.7867
Epoch

## Classfication Model on 600 * 600 Model

In [8]:
efficient_net_b7_cf_base_model = tfhb.KerasLayer(efficient_net_b7_cf_link,
                                                 trainable=False)

In [10]:
get_model_information(efficient_net_b7_cf_base_model)

Thickness of the model: 1037
Layer                                                                            Shape                Type      
efficientnet-b7/stem/conv2d/kernel:0                                             (3, 3, 3, 64)        float32   
efficientnet-b7/stem/tpu_batch_normalization/gamma:0                             (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/beta:0                              (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/moving_mean:0                       (64,)                float32   
efficientnet-b7/stem/tpu_batch_normalization/moving_variance:0                   (64,)                float32   
efficientnet-b7/blocks_0/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 64, 1)        float32   
efficientnet-b7/blocks_0/tpu_batch_normalization/gamma:0                         (64,)                float32   
efficientnet-b7/blocks_0/tpu_batch_normalization/beta:0            

efficientnet-b7/blocks_6/depthwise_conv2d/depthwise_kernel:0                     (3, 3, 288, 1)       float32   
efficientnet-b7/blocks_6/tpu_batch_normalization_1/gamma:0                       (288,)               float32   
efficientnet-b7/blocks_6/tpu_batch_normalization_1/beta:0                        (288,)               float32   
efficientnet-b7/blocks_6/tpu_batch_normalization_1/moving_mean:0                 (288,)               float32   
efficientnet-b7/blocks_6/tpu_batch_normalization_1/moving_variance:0             (288,)               float32   
efficientnet-b7/blocks_6/se/conv2d/kernel:0                                      (1, 1, 288, 12)      float32   
efficientnet-b7/blocks_6/se/conv2d/bias:0                                        (12,)                float32   
efficientnet-b7/blocks_6/se/conv2d_1/kernel:0                                    (1, 1, 12, 288)      float32   
efficientnet-b7/blocks_6/se/conv2d_1/bias:0                                      (288,)         

efficientnet-b7/blocks_11/tpu_batch_normalization_1/moving_variance:0            (288,)               float32   
efficientnet-b7/blocks_11/se/conv2d/kernel:0                                     (1, 1, 288, 12)      float32   
efficientnet-b7/blocks_11/se/conv2d/bias:0                                       (12,)                float32   
efficientnet-b7/blocks_11/se/conv2d_1/kernel:0                                   (1, 1, 12, 288)      float32   
efficientnet-b7/blocks_11/se/conv2d_1/bias:0                                     (288,)               float32   
efficientnet-b7/blocks_11/conv2d_1/kernel:0                                      (1, 1, 288, 80)      float32   
efficientnet-b7/blocks_11/tpu_batch_normalization_2/gamma:0                      (80,)                float32   
efficientnet-b7/blocks_11/tpu_batch_normalization_2/beta:0                       (80,)                float32   
efficientnet-b7/blocks_11/tpu_batch_normalization_2/moving_mean:0                (80,)          

efficientnet-b7/blocks_15/conv2d_1/kernel:0                                      (1, 1, 480, 80)      float32   
efficientnet-b7/blocks_15/tpu_batch_normalization_2/gamma:0                      (80,)                float32   
efficientnet-b7/blocks_15/tpu_batch_normalization_2/beta:0                       (80,)                float32   
efficientnet-b7/blocks_15/tpu_batch_normalization_2/moving_mean:0                (80,)                float32   
efficientnet-b7/blocks_15/tpu_batch_normalization_2/moving_variance:0            (80,)                float32   
efficientnet-b7/blocks_16/conv2d/kernel:0                                        (1, 1, 80, 480)      float32   
efficientnet-b7/blocks_16/tpu_batch_normalization/gamma:0                        (480,)               float32   
efficientnet-b7/blocks_16/tpu_batch_normalization/beta:0                         (480,)               float32   
efficientnet-b7/blocks_16/tpu_batch_normalization/moving_mean:0                  (480,)         

efficientnet-b7/blocks_22/tpu_batch_normalization_1/moving_variance:0            (960,)               float32   
efficientnet-b7/blocks_22/se/conv2d/kernel:0                                     (1, 1, 960, 40)      float32   
efficientnet-b7/blocks_22/se/conv2d/bias:0                                       (40,)                float32   
efficientnet-b7/blocks_22/se/conv2d_1/kernel:0                                   (1, 1, 40, 960)      float32   
efficientnet-b7/blocks_22/se/conv2d_1/bias:0                                     (960,)               float32   
efficientnet-b7/blocks_22/conv2d_1/kernel:0                                      (1, 1, 960, 160)     float32   
efficientnet-b7/blocks_22/tpu_batch_normalization_2/gamma:0                      (160,)               float32   
efficientnet-b7/blocks_22/tpu_batch_normalization_2/beta:0                       (160,)               float32   
efficientnet-b7/blocks_22/tpu_batch_normalization_2/moving_mean:0                (160,)         

efficientnet-b7/blocks_28/tpu_batch_normalization_1/moving_variance:0            (960,)               float32   
efficientnet-b7/blocks_28/se/conv2d/kernel:0                                     (1, 1, 960, 40)      float32   
efficientnet-b7/blocks_28/se/conv2d/bias:0                                       (40,)                float32   
efficientnet-b7/blocks_28/se/conv2d_1/kernel:0                                   (1, 1, 40, 960)      float32   
efficientnet-b7/blocks_28/se/conv2d_1/bias:0                                     (960,)               float32   
efficientnet-b7/blocks_28/conv2d_1/kernel:0                                      (1, 1, 960, 224)     float32   
efficientnet-b7/blocks_28/tpu_batch_normalization_2/gamma:0                      (224,)               float32   
efficientnet-b7/blocks_28/tpu_batch_normalization_2/beta:0                       (224,)               float32   
efficientnet-b7/blocks_28/tpu_batch_normalization_2/moving_mean:0                (224,)         

efficientnet-b7/blocks_34/tpu_batch_normalization_2/moving_mean:0                (224,)               float32   
efficientnet-b7/blocks_34/tpu_batch_normalization_2/moving_variance:0            (224,)               float32   
efficientnet-b7/blocks_35/conv2d/kernel:0                                        (1, 1, 224, 1344)    float32   
efficientnet-b7/blocks_35/tpu_batch_normalization/gamma:0                        (1344,)              float32   
efficientnet-b7/blocks_35/tpu_batch_normalization/beta:0                         (1344,)              float32   
efficientnet-b7/blocks_35/tpu_batch_normalization/moving_mean:0                  (1344,)              float32   
efficientnet-b7/blocks_35/tpu_batch_normalization/moving_variance:0              (1344,)              float32   
efficientnet-b7/blocks_35/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 1344, 1)      float32   
efficientnet-b7/blocks_35/tpu_batch_normalization_1/gamma:0                      (1344,)        

efficientnet-b7/blocks_40/tpu_batch_normalization_2/moving_variance:0            (384,)               float32   
efficientnet-b7/blocks_41/conv2d/kernel:0                                        (1, 1, 384, 2304)    float32   
efficientnet-b7/blocks_41/tpu_batch_normalization/gamma:0                        (2304,)              float32   
efficientnet-b7/blocks_41/tpu_batch_normalization/beta:0                         (2304,)              float32   
efficientnet-b7/blocks_41/tpu_batch_normalization/moving_mean:0                  (2304,)              float32   
efficientnet-b7/blocks_41/tpu_batch_normalization/moving_variance:0              (2304,)              float32   
efficientnet-b7/blocks_41/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 2304, 1)      float32   
efficientnet-b7/blocks_41/tpu_batch_normalization_1/gamma:0                      (2304,)              float32   
efficientnet-b7/blocks_41/tpu_batch_normalization_1/beta:0                       (2304,)        

efficientnet-b7/blocks_46/tpu_batch_normalization_2/beta:0                       (384,)               float32   
efficientnet-b7/blocks_46/tpu_batch_normalization_2/moving_mean:0                (384,)               float32   
efficientnet-b7/blocks_46/tpu_batch_normalization_2/moving_variance:0            (384,)               float32   
efficientnet-b7/blocks_47/conv2d/kernel:0                                        (1, 1, 384, 2304)    float32   
efficientnet-b7/blocks_47/tpu_batch_normalization/gamma:0                        (2304,)              float32   
efficientnet-b7/blocks_47/tpu_batch_normalization/beta:0                         (2304,)              float32   
efficientnet-b7/blocks_47/tpu_batch_normalization/moving_mean:0                  (2304,)              float32   
efficientnet-b7/blocks_47/tpu_batch_normalization/moving_variance:0              (2304,)              float32   
efficientnet-b7/blocks_47/depthwise_conv2d/depthwise_kernel:0                    (5, 5, 2304, 1)

efficientnet-b7/blocks_52/tpu_batch_normalization_1/moving_mean:0                (3840,)              float32   
efficientnet-b7/blocks_52/tpu_batch_normalization_1/moving_variance:0            (3840,)              float32   
efficientnet-b7/blocks_52/se/conv2d/kernel:0                                     (1, 1, 3840, 160)    float32   
efficientnet-b7/blocks_52/se/conv2d/bias:0                                       (160,)               float32   
efficientnet-b7/blocks_52/se/conv2d_1/kernel:0                                   (1, 1, 160, 3840)    float32   
efficientnet-b7/blocks_52/se/conv2d_1/bias:0                                     (3840,)              float32   
efficientnet-b7/blocks_52/conv2d_1/kernel:0                                      (1, 1, 3840, 640)    float32   
efficientnet-b7/blocks_52/tpu_batch_normalization_2/gamma:0                      (640,)               float32   
efficientnet-b7/blocks_52/tpu_batch_normalization_2/beta:0                       (640,)         

In [11]:
efficient_net_b7_cf_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b7_cf_base_model,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b7_cf_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [12]:
efficient_net_b7_cf_model.fit(train_ds_mn_600,
                              epochs=10,
                              steps_per_epoch=len(train_ds_mn_600),
                              validation_data=valid_ds_mn_600,
                              validation_steps=len(valid_ds_mn_600))

Epoch 1/10
22/22 [==============================] - 56s 2s/step - loss: 1.6243 - accuracy: 0.4711 - val_loss: 0.9429 - val_accuracy: 0.7467
Epoch 2/10
22/22 [==============================] - 38s 2s/step - loss: 0.6205 - accuracy: 0.8385 - val_loss: 0.5925 - val_accuracy: 0.8267
Epoch 3/10
22/22 [==============================] - 39s 2s/step - loss: 0.4172 - accuracy: 0.9007 - val_loss: 0.5314 - val_accuracy: 0.8267
Epoch 4/10
22/22 [==============================] - 40s 2s/step - loss: 0.3389 - accuracy: 0.9126 - val_loss: 0.4664 - val_accuracy: 0.8667
Epoch 5/10
22/22 [==============================] - 41s 2s/step - loss: 0.2939 - accuracy: 0.9259 - val_loss: 0.4509 - val_accuracy: 0.8400
Epoch 6/10
22/22 [==============================] - 42s 2s/step - loss: 0.2410 - accuracy: 0.9378 - val_loss: 0.4349 - val_accuracy: 0.8667
Epoch 7/10
22/22 [==============================] - 42s 2s/step - loss: 0.2170 - accuracy: 0.9481 - val_loss: 0.4308 - val_accuracy: 0.8667
Epoch 8/10
22/22 [==

## Classification Model On 224 * 224 Images

In [13]:
efficient_net_b7_cf_base_model_fg = tfhb.KerasLayer(efficient_net_b7_cf_link,
                                                    trainable=False)

In [14]:
efficient_net_b7_cf_model_fg = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b7_cf_base_model_fg,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b7_cf_model_fg.compile(loss="categorical_crossentropy",
                                     optimizer=tf.keras.optimizers.Adam(),
                                     metrics=["accuracy"])

In [16]:
efficient_net_b7_cf_model_fg.fit(train_ds_mn_224,
                                 epochs=10,
                                 steps_per_epoch=len(train_ds_mn_224),
                                 validation_data=valid_ds_mn_224,
                                 validation_steps=len(valid_ds_mn_224))

Epoch 1/10
22/22 [==============================] - 19s 422ms/step - loss: 1.8877 - accuracy: 0.3911 - val_loss: 1.2082 - val_accuracy: 0.6400
Epoch 2/10
22/22 [==============================] - 6s 274ms/step - loss: 0.7666 - accuracy: 0.7600 - val_loss: 0.9297 - val_accuracy: 0.7067
Epoch 3/10
22/22 [==============================] - 6s 276ms/step - loss: 0.5319 - accuracy: 0.8459 - val_loss: 0.8976 - val_accuracy: 0.7200
Epoch 4/10
22/22 [==============================] - 6s 274ms/step - loss: 0.4320 - accuracy: 0.8667 - val_loss: 0.8669 - val_accuracy: 0.7733
Epoch 5/10
22/22 [==============================] - 6s 274ms/step - loss: 0.3723 - accuracy: 0.9052 - val_loss: 0.8697 - val_accuracy: 0.7733
Epoch 6/10
22/22 [==============================] - 6s 276ms/step - loss: 0.3109 - accuracy: 0.9244 - val_loss: 0.8677 - val_accuracy: 0.7733
Epoch 7/10
22/22 [==============================] - 6s 278ms/step - loss: 0.2731 - accuracy: 0.9467 - val_loss: 0.8543 - val_accuracy: 0.7867
Epoch

# Using Model Save CheckPoint Callback

In [34]:
def create_modelsave_callback(dir_name,experiment_name):
    file_path = dir_name + "//" + experiment_name + "//" + "_{epoch:02d}.ckpt"
    modelsave_callback_obj = tf.keras.callbacks.ModelCheckpoint(file_path,
                                                                verbose=1,
                                                                save_weights_only=True,
                                                                save_freq="epoch")
    return modelsave_callback_obj

In [35]:
efficient_net_b0_fv_base_model = tfhb.KerasLayer(efficient_net_b0_fv_link,
                                                 trainable=False)

In [36]:
efficient_net_b0_fv_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b0_fv_base_model,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b0_fv_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [37]:
efficient_net_b0_fv_model.fit(train_ds_mn_224,
                              epochs=10,
                              steps_per_epoch=len(train_ds_mn_224),
                              validation_data=valid_ds_mn_224,
                              validation_steps=len(valid_ds_mn_224),
                              callbacks=[create_modelsave_callback(dir_name="TFSave",
                                                                     experiment_name="Efficient_Net_fv_b0")])

Epoch 1/10
21/22 [===========================>..] - ETA: 0s - loss: 1.9763 - accuracy: 0.3557
Epoch 1: saving model to TFSave//Efficient_Net_fv_b0\_01.ckpt
22/22 [==============================] - 6s 128ms/step - loss: 1.9755 - accuracy: 0.3570 - val_loss: 1.6097 - val_accuracy: 0.5600
Epoch 2/10
21/22 [===========================>..] - ETA: 0s - loss: 1.1465 - accuracy: 0.7664
Epoch 2: saving model to TFSave//Efficient_Net_fv_b0\_02.ckpt
22/22 [==============================] - 2s 93ms/step - loss: 1.1479 - accuracy: 0.7659 - val_loss: 1.1666 - val_accuracy: 0.6267
Epoch 3/10
21/22 [===========================>..] - ETA: 0s - loss: 0.8099 - accuracy: 0.8333
Epoch 3: saving model to TFSave//Efficient_Net_fv_b0\_03.ckpt
22/22 [==============================] - 2s 101ms/step - loss: 0.8120 - accuracy: 0.8326 - val_loss: 0.9752 - val_accuracy: 0.7467
Epoch 4/10
21/22 [===========================>..] - ETA: 0s - loss: 0.6455 - accuracy: 0.8616
Epoch 4: saving model to TFSave//Efficient_Net

In [39]:
efficient_net_b0_fv_model.load_weights("TFSave//Efficient_Net_fv_b0\_10.ckpt")

In [40]:
efficient_net_b0_fv_model.evaluate(valid_ds_mn_224)

3/3 [==============================] - 0s 49ms/step - loss: 0.6875 - accuracy: 0.7733


[0.6875088214874268, 0.7733333110809326]

In [41]:
efficient_net_b0_fv_model.evaluate(train_ds_mn_224)

22/22 [==============================] - 1s 52ms/step - loss: 0.2887 - accuracy: 0.9511


[0.28871065378189087, 0.9511111378669739]

In [42]:
efficient_net_b0_fv_model.load_weights("TFSave//Efficient_Net_fv_b0\_05.ckpt")

In [43]:
efficient_net_b0_fv_model.evaluate(valid_ds_mn_224)

3/3 [==============================] - 0s 48ms/step - loss: 0.8096 - accuracy: 0.7600


[0.809586763381958, 0.7599999904632568]

In [44]:
efficient_net_b0_fv_model.evaluate(train_ds_mn_224)

22/22 [==============================] - 2s 52ms/step - loss: 0.4958 - accuracy: 0.9052


[0.4957684576511383, 0.9051851630210876]

In [65]:
def create_modelsave_callback(dir_name,experiment_name):
    file_path = dir_name + "//" + experiment_name + "//" + ".ckpt"
    modelsave_callback_obj = tf.keras.callbacks.ModelCheckpoint(file_path,
                                                                verbose=1,
                                                                save_weights_only=True,
                                                                save_best_only=True,
                                                                save_freq="epoch")
    return modelsave_callback_obj

In [66]:
efficient_net_b0_fv_base_model = tfhb.KerasLayer(efficient_net_b0_fv_link,
                                                 trainable=False)

In [67]:
efficient_net_b0_fv_model = tf.keras.models.Sequential([tf.keras.layers.Rescaling(1.0/255.0),
                                                        efficient_net_b0_fv_base_model,
                                                        tf.keras.layers.Dense(10,activation="softmax")])

efficient_net_b0_fv_model.compile(loss="categorical_crossentropy",
                                  optimizer=tf.keras.optimizers.Adam(),
                                  metrics=["accuracy"])

In [68]:
efficient_net_b0_fv_model.fit(train_ds_mn_224,
                              epochs=50,
                              steps_per_epoch=len(train_ds_mn_224),
                              validation_data=valid_ds_mn_224,
                              validation_steps=len(valid_ds_mn_224),
                              callbacks=[create_modelsave_callback(dir_name="TFSaveBest",
                                                                     experiment_name="Efficient_Net_fv_b0")])

Epoch 1/50
21/22 [===========================>..] - ETA: 0s - loss: 2.0199 - accuracy: 0.3170
Epoch 1: val_loss improved from inf to 1.58804, saving model to TFSaveBest//Efficient_Net_fv_b0\.ckpt
22/22 [==============================] - 6s 145ms/step - loss: 2.0196 - accuracy: 0.3170 - val_loss: 1.5880 - val_accuracy: 0.6000
Epoch 2/50
22/22 [==============================] - ETA: 0s - loss: 1.1698 - accuracy: 0.7556
Epoch 2: val_loss improved from 1.58804 to 1.14431, saving model to TFSaveBest//Efficient_Net_fv_b0\.ckpt
22/22 [==============================] - 2s 91ms/step - loss: 1.1698 - accuracy: 0.7556 - val_loss: 1.1443 - val_accuracy: 0.7467
Epoch 3/50
22/22 [==============================] - ETA: 0s - loss: 0.8205 - accuracy: 0.8193
Epoch 3: val_loss improved from 1.14431 to 0.96230, saving model to TFSaveBest//Efficient_Net_fv_b0\.ckpt
22/22 [==============================] - 2s 89ms/step - loss: 0.8205 - accuracy: 0.8193 - val_loss: 0.9623 - val_accuracy: 0.7867
Epoch 4/50
22

Epoch 27/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0992 - accuracy: 0.9970
Epoch 27: val_loss did not improve from 0.63867
22/22 [==============================] - 1s 60ms/step - loss: 0.0988 - accuracy: 0.9970 - val_loss: 0.6471 - val_accuracy: 0.8267
Epoch 28/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0931 - accuracy: 0.9970
Epoch 28: val_loss did not improve from 0.63867
22/22 [==============================] - 1s 59ms/step - loss: 0.0929 - accuracy: 0.9970 - val_loss: 0.6532 - val_accuracy: 0.8267
Epoch 29/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0885 - accuracy: 0.9985
Epoch 29: val_loss did not improve from 0.63867
22/22 [==============================] - 1s 59ms/step - loss: 0.0883 - accuracy: 0.9985 - val_loss: 0.6504 - val_accuracy: 0.8133
Epoch 30/50
21/22 [===========================>..] - ETA: 0s - loss: 0.0838 - accuracy: 0.9985
Epoch 30: val_loss did not improve from 0.63867
22/22 [==============================]

In [69]:
efficient_net_b0_fv_model.load_weights("TFSaveBest//Efficient_Net_fv_b0\.ckpt")

In [70]:
efficient_net_b0_fv_model.evaluate(valid_ds_mn_224)

3/3 [==============================] - 0s 46ms/step - loss: 0.6387 - accuracy: 0.8267


[0.6386730670928955, 0.8266666531562805]